In [32]:
import pandas as pd 
import json
from azure.storage.blob import BlobServiceClient
from pyspark.sql.types import *
import re

In [33]:
blob_path='data.medicaid.gov/api/1/datastore/query/'

In [35]:
BLOB_ACCOUNT_NAME = 'usafactsbronze'
LINKED_SERVICE_NAME = 'bronze'
input_container_name='bronze'
BLOB_SAS_TOKEN = mssparkutils.credentials.getConnectionStringOrCreds(LINKED_SERVICE_NAME)
source_blob_service_client = BlobServiceClient("https://{}.blob.core.windows.net".format(BLOB_ACCOUNT_NAME), credential=BLOB_SAS_TOKEN)
container_client = source_blob_service_client.get_container_client(input_container_name)
blobs = container_client.list_blobs(name_starts_with=blob_path)

In [36]:
# Azure storage access info 
blob_account_name = 'usafactssilver'
blob_container_name = 'silver'
linked_service_name = 'silver' 
blob_sas_token = mssparkutils.credentials.getConnectionStringOrCreds(linked_service_name) 
# Allow SPARK to access from Blob remotely
spark.conf.set('fs.azure.sas.%s.%s.blob.core.windows.net' % (blob_container_name, blob_account_name), blob_sas_token) 

In [37]:
for blob in blobs:
    try:
        blob_client = container_client.get_blob_client(blob) 
        blob_data = blob_client.download_blob().readall()
        json_data = json.loads(blob_data)
        table_data=json_data['results']
        df=pd.DataFrame(table_data)
        spark_schema = StructType([StructField(col, StringType(), True) for col in df.columns])
        spark_df = spark.createDataFrame(df, schema=spark_schema)
        special_characters = r'[^a-zA-Z0-9-_/.]'
        file_path = re.sub(special_characters, '_', blob.name)
        wasbs_path = 'wasbs://%s@%s.blob.core.windows.net/%s' % (blob_container_name, blob_account_name,file_path) 
        spark_df.write.format('delta').mode('overwrite').option("overwriteSchema",True).option("path",wasbs_path).save()
        print('file_uploaded:',file_path)
    except Exception as e:
        print(f'error:{blob.name}',str(e))